In [ ]:
import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns
import pandas as pd
import numpy as np
import json
import ast

## Data Collection

In [ ]:
rounds_df = pd.read_csv("../data/rounds.csv", index_col=[0])
rounds_df["date"] = pd.to_datetime(rounds_df["date"], infer_datetime_format=True)
rounds_df.head()

In [ ]:
players_df = pd.read_csv("../data/players.csv", index_col=[0])
players_df.head()

In [ ]:
transfers_df = pd.read_csv("../data/transfers.csv", index_col=[0])
transfers_df["date"] = pd.to_datetime(transfers_df["date"], infer_datetime_format=True)
transfers_df = transfers_df.drop(columns=["money"]).sort_values("date").reset_index(drop=True)
transfers_df.head()

## Data Integration

In [ ]:
transfers = []
index_blacklist = []
for idx, row in transfers_df.iterrows():
    player = row.player_name
    if idx in index_blacklist or player not in players_df.name.values:
        continue
    
    # Info about the player
    player_info = players_df[players_df.name == player].iloc[0]
    
    # Time series of the value of the player in time
    value_df = pd.read_csv("../data/player_values/{0}.csv".format(player), sep=";", 
                           names=["date", "market_value"], skiprows=1)
    
    value_df["date"] = value_df.date.apply(lambda x: x.split("+")[0])
    value_df["date"] = pd.to_datetime(value_df["date"], format="%a %b %d %Y %H:%M:%S %Z")
    
    # Iterations
    current_idx = idx
    keep_looking = True
    purchase_date = row.date
    purchase_price = row.value
    origin_user = row.origin_user
    destination_user = row.destination_user

    while keep_looking: 
        next_sale = transfers_df[(transfers_df.index > current_idx) & 
                                 (transfers_df.player_name == player) & 
                                 (transfers_df.origin_user == destination_user)]
        
        purchase_market_value = value_df[(value_df.date <= purchase_date)].sort_values("date", ascending=False).iloc[0].market_value
        
        if next_sale.shape[0] > 0:
            sold_to = next_sale.iloc[0].destination_user
            sold_by = next_sale.iloc[0].origin_user
            sale_price = next_sale.iloc[0].value
            sale_date = next_sale.iloc[0].date  
            
            sale_market_value = value_df[(value_df.date <= sale_date)].sort_values("date", ascending=False).iloc[0].market_value
            fixtures_in_team = rounds_df.loc[(rounds_df.date >= purchase_date) & (rounds_df.date <= sale_date), "fixture"].tolist()
        else:
            sale_market_value = value_df.sort_values("date", ascending=False).iloc[0].market_value
            sale_price = None
            sale_date = None
            sold_to = None
            sold_by = None
            
            fixtures_in_team = rounds_df.loc[(rounds_df.date >= purchase_date), "fixture"].tolist()
        
        # Compute points
        idx_rounds = [round_idx for round_idx, r in enumerate(ast.literal_eval(player_info.rounds)) if r in fixtures_in_team]
        value_rounds = [v for round_idx, v in enumerate(ast.literal_eval(player_info.points)) if round_idx in idx_rounds]
                
        points = 0
        num_matches = 0
        num_played_matches = 0
        for round_idx in range(len(idx_rounds)):
            num_matches += 1
            try:
                points += int(value_rounds[round_idx])
                num_played_matches += 1
            except ValueError:
                continue
        
        index_blacklist.append(current_idx)
        transfers.append({
            "player": player,
            "team": player_info["team"],
            "position": player_info["position"],

            "purchase_market_value": purchase_market_value,
            "purchase_price": purchase_price,
            "purchased_by": destination_user,
            "purchase_date": purchase_date,
            "purchased_from": origin_user,

            "sale_market_value": sale_market_value,
            "sale_price": sale_price,
            "sale_date": sale_date,
            "sold_by": sold_by,
            "sold_to": sold_to,

            "num_played_matches": num_played_matches,
            "num_matches": num_matches,
            "points": points
        })
        
        if sold_to:
            current_idx = next_sale.index[0]
            origin_user = sold_by
            destination_user = sold_to
            purchase_date = sale_date
            purchase_price = sale_price
        else:
            keep_looking = False

df = pd.DataFrame(transfers)
df = df[df.purchased_by.notna()]

In [ ]:
df["price_diff"] = df.apply(lambda x: x.purchase_price-x.purchase_market_value, axis=1)
df["rel_price_diff"] = df.apply(lambda x: x.price_diff/x.purchase_price*100, axis=1)
df["profit"] = df.apply(lambda x: (x.sale_price if not np.isnan(x.sale_price) else x.sale_market_value)-x.purchase_price, axis=1)
df["point_avg"] = df.apply(lambda x: x.points/x.num_matches if x.num_matches > 0 else 0, axis=1)
df["effective_point_avg"] = df.apply(lambda x: x.points/x.num_played_matches if x.num_played_matches > 0 else 0, axis=1)
df["price_per_point"] = df.apply(lambda x: x.purchase_price/x.points if x.points > 0 else None, axis=1)
df["ROI"] = df.apply(lambda x: x.profit/x.purchase_price if x.purchase_price > 0 else None, axis=1)

In [ ]:
df.sort_values("purchase_date")

# Results

In [ ]:
th_props = [
  ('font-size', '11px'),
  ('text-align', 'center'),
  ('font-weight', 'bold'),
  ('color', '#6d6d6d'),
  ('background-color', '#f7f7f9')
]

td_props = [
  ('font-size', '11px')
]

styles = [
  dict(selector="th", props=th_props),
  dict(selector="td", props=td_props)
]

def human_format(num):
    num = float('{:.3g}'.format(num))
    magnitude = 0
    while abs(num) >= 1000:
        magnitude += 1
        num /= 1000.0
    return '{}{}'.format('{:f}'.format(num).rstrip('0').rstrip('.'), ['', 'K', 'M', 'B', 'T'][magnitude])

In [ ]:
team_df = df[df.sale_price.isna()].groupby("purchased_by").agg({"profit": "sum"}).reset_index()
sales_df = df[df.sale_price.notna()].groupby("purchased_by").agg({"profit": "sum"}).reset_index()

teams = df.purchased_by.unique()
num_transfers = [df[(df.purchased_by == team)].shape[0] for team in teams]
profit_team = [team_df.loc[team_df.purchased_by == team, "profit"].iloc[0] for team in teams]
profit_sales = [sales_df.loc[sales_df.purchased_by == team, "profit"].iloc[0] for team in teams]

# Make the plot
fig, ax = plt.subplots(figsize=(12, 12), dpi=180)
plt.scatter(profit_sales, profit_team, s=[x**2/10 for x in num_transfers], alpha=0.7)

for i, team in enumerate(teams):
    ax.annotate(team, (profit_sales[i], profit_team[i]), xytext=(0, num_transfers[i]/1.5), textcoords='offset pixels',
                horizontalalignment='center', verticalalignment='center')

# Add xticks on the middle of the group bars
plt.ylabel('Team profit', fontweight='bold', color="grey", labelpad=10, rotation=0)
plt.xlabel('Sales profit', fontweight='bold', color="grey", labelpad=10)

# Removing top and right borders
ax.set_facecolor('white')
fig.patch.set_alpha(1)

ax.tick_params(colors='grey')
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['left'].set_color("grey")
ax.spines['bottom'].set_color("grey")

xmax = 20000001
xmin = -20000000
xrange = range(xmin, xmax, 5000000)
plt.xticks(xrange, [human_format(x) for x in xrange])
plt.xlim(xmin, xmax)

ymin = 0+5000000
ymax = 40000000+5000000
yrange = range(ymin, ymax, 5000000)
plt.yticks(yrange, [human_format(y) for y in yrange])
plt.ylim(0, ymax-5000000)

ax.yaxis.set_label_coords(0.5, 1.03)

ax.spines['left'].set_position(('data', 0))
ax.spines['bottom'].set_position(('data', 0))
    
for ntransfers in [30, 60, 90]:
    plt.scatter([], [], c="#1f77b4", alpha=0.7, s=ntransfers**2/10, label=str(ntransfers)+" transfers")
    
leg = plt.legend(scatterpoints=1, frameon=False, labelspacing=2)
for text in leg.get_texts():
    plt.setp(text, color='grey')

# Showing
plt.show()

In [ ]:
top = df.loc[df.points > 75, ["player", "purchased_by", "points", "price_per_point"]].sort_values("price_per_point").head(5)
top.rename(columns={"player": "Player", "purchased_by": "Team", "points": "Points", "price_per_point": "Price per point"}, inplace=True)

top["Price per point"] = top["Price per point"].apply(human_format)

(top.style
    .applymap(lambda x: "font-weight: bold", subset=['Player'])
    .set_caption('Most efficient players (> 75 points)')
    .hide_index()
    .set_table_styles(styles))

In [ ]:
top = df[["player", "purchased_by", "purchase_price", "profit"]].sort_values("profit", ascending=False).head(5)
top.rename(columns={"player": "Player", "purchased_by": "Team", "purchase_price": "Purchase price", "profit": "Profit"}, inplace=True)

top["Purchase price"] = top["Purchase price"].apply(human_format)
top["Profit"] = top["Profit"].apply(human_format)

(top.style
    .applymap(lambda x: "font-weight: bold", subset=['Player'])
    .set_caption('Most profitable transfers')
    .hide_index()
    .set_table_styles(styles))

In [ ]:
top = df.loc[df.purchase_price > 1000000, ["player", "purchased_by", "purchase_price", "ROI"]].sort_values("ROI", ascending=False).head(5)
top.rename(columns={"player": "Player", "purchased_by": "Team", "purchase_price": "Purchase price"}, inplace=True)

top["Purchase price"] = top["Purchase price"].apply(human_format)

(top.style
    .applymap(lambda x: "font-weight: bold", subset=['Player'])
    .format({'ROI': "{:.0%}"})
    .set_caption('Most profitable transfers based on ROI (> 1M)')
    .hide_index()
    .set_table_styles(styles))

In [ ]:
top = df[["player", "purchased_by", "purchase_price", "profit"]].sort_values("profit").head(5)
top.rename(columns={"player": "Player", "purchased_by": "Team", "purchase_price": "Purchase price", "profit": "Profit"}, inplace=True)

top["Purchase price"] = top["Purchase price"].apply(human_format)
top["Profit"] = top["Profit"].apply(human_format)

(top.style
    .applymap(lambda x: "font-weight: bold", subset=['Player'])
    .set_caption('Least profitable transfers')
    .hide_index()
    .set_table_styles(styles))

In [ ]:
top = df.loc[df.purchase_price > 2000000, ["player", "purchased_by", "purchase_price", "ROI"]].sort_values("ROI").head(5)
top.rename(columns={"player": "Player", "purchased_by": "Team", "purchase_price": "Purchase price"}, inplace=True)

top["Purchase price"] = top["Purchase price"].apply(human_format)

(top.style
    .applymap(lambda x: "font-weight: bold", subset=['Player'])
    .format({'ROI': "{:.0%}"})
    .set_caption('Least profitable transfers based on ROI (> 2M)')
    .hide_index()
    .set_table_styles(styles))

In [ ]:
fig = px.violin(df, y="profit", x="purchased_by", points="all", hover_name="player")
fig.update_yaxes(title_text='Profit')
fig.update_xaxes(title_text='Team')
fig.show()